# Formatted Zone

In [6]:
# Uncomment to upgrade packages
# !pip install duckdb --pre --upgrade
# % pip install pandas --upgrade --user --quiet
# % pip install xlrd 

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.5/96.5 KB 404.7 kB/s eta 0:00:00 kB/s eta 0:00:01:01


In [2]:
import duckdb
import pandas as pd

In [3]:
data = pd.read_csv('landing/temporal/madrid.csv', sep=';',header=0, encoding='latin1')
data

,municipio_codigo,municipio_nombre,distrito_codigo,distrito_nombre,barrio_codigo,barrio_nombre,sexo,nacionalidad,poblacion_empadronada
0,79,Madrid,79601,Centro,796011,Palacio,Hombre,Africa,163
1,79,Madrid,79601,Centro,796011,Palacio,Hombre,América Central y del Caribe,83
2,79,Madrid,79601,Centro,796011,Palacio,Hombre,América del Norte,169
3,79,Madrid,79601,Centro,796011,Palacio,Hombre,América del Sur,384
4,79,Madrid,79601,Centro,796011,Palacio,Hombre,Apátridas,1
...,...,...,...,...,...,...,...,...,...
2555,79,Madrid,79621,Barajas,796215,Corralejos,Mujer,Asia,14
2556,79,Madrid,79621,Barajas,796215,Corralejos,Mujer,Oceanía,0
2557,79,Madrid,79621,Barajas,796215,Corralejos,Mujer,Otros países de América,0
2558,79,Madrid,79621,Barajas,796215,Corralejos,Mujer,Resto de Europa,9


In [10]:
household = pd.read_excel('landing/temporal/C14150219.xls',sheet_name='Composicion del hogar',header=[0,1])
household

Ciudad de Madrid  \
     Explotación estadística del Padrón Municipal de Habitantes revisado a 1 de enero de 2019   
0                                                   NaN                                         
1     Hogares por Composición del hogar según Distri...                                         
2                                    Distrito / Sección                                         
3                                                   NaN                                         
4                                                   NaN                                         
...                                                 ...                                         
2488                                                NaN                                         
2489                                                NaN                                         
2490                                                NaN                                         
2491                                                NaN                                         
2492  FUENTE: Padrón Municipal de habitantes. Subdir...                                         

                                                                                                 \
     Explotación estadística del Padrón Municipal de Habitantes revisado a 1 de enero de 2019.1   
0                                                   NaN                                           
1                                                   NaN                                           
2                                                   NaN                                           
3                                                   NaN                                           
4                                                   NaN                                           
...                                                 ...                                           
2488                                            21031.0                                           
2489                                            21032.0                                           
2490                                            21033.0                                           
2491                                                NaN                                           
2492                                                NaN                                           

                                                                                                 \
     Explotación estadística del Padrón Municipal de Habitantes revisado a 1 de enero de 2019.2   
0                                                   NaN                                           
1                                                   NaN                                           
2                                                 Total                                           
3                                                   NaN                                           
4                                                   NaN                                           
...                                                 ...                                           
2488                                                948                                           
2489                                                658                                           
2490                                                907                                           
2491                                                NaN                                           
2492                                                NaN                                           

                                                                                                 \
     Explotación estadística del Padrón Municipal de Habitantes revisado a 1 de enero de 2019.3   
0                                        

In [ ]:
# connect to an in-memory temporary database
con = duckdb.connect()
con.register("madrid_df", data)
con.execute("SELECT municipio_codigo FROM madrid_df WHERE distrito_nombre = 'Arganzuela'").fetchdf()

,municipio_codigo
0,79
1,79
2,79
3,79
4,79
...,...
135,79
136,79
137,79
138,79
